<a href="https://colab.research.google.com/github/SherZhou/HW_2/blob/main/H2O_autoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the H2O library and start up the H2O cluter locally on your machine
# import h2o
# Load the H2O library and start up the H2O cluter locally on your machine
!pip install h2o
import h2o

# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)
# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 58 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_s8n4fp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.042 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 58 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_s8n4fp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.042 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [ ]:
from google.colab import drive
# import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loan_csv = '/content/drive/MyDrive/notebook/big data/data.csv'  # modify this for your machine
data = h2o.import_file(loan_csv)  # 163,987 rows x 15 columns

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
data

In [ ]:
data.shape


In [ ]:
import multiprocessing
multiprocessing.cpu_count()

12

In [ ]:
# data['bad_loan'] = data['bad_loan'].asfactor()  #encode the binary repsonse as a factor

In [ ]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]
print (train.nrow,valid.nrow,test.nrow)

350181 74945 74874


In [ ]:
y = 'class'
x = list(data.columns)

In [ ]:
x.remove(y)  #remove the response

In [ ]:
x

['objid',
 'ra',
 'dec',
 'u',
 'g',
 'r',
 'i',
 'z',
 'run',
 'rerun',
 'camcol',
 'field',
 'modelFlux_u',
 'modelFlux_g',
 'modelFlux_r',
 'modelFlux_i',
 'modelFlux_z',
 'specobjid',
 'redshift',
 'plate',
 'mjd',
 'fiberid']

In [ ]:
import requests
import json

# 获取自己IP
res = requests.get("https://searchplugin.csdn.net/api/v1/ip/get?")
res = res.text


In [ ]:
res

'{"code":200,"msg":"success","data":{"address":"新加坡 新加坡  cloud.google.com","ip":"34.87.83.254"}}'

# AUTO ML

In [ ]:

from h2o.automl import H2OAutoML


# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=25, seed=1)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

# model_id                                                  auc    logloss    mean_per_class_error      rmse       mse
# ---------------------------------------------------  --------  ---------  ----------------------  --------  --------
# StackedEnsemble_AllModels_AutoML_20181212_105540     0.789801   0.551109                0.333174  0.43211   0.186719
# StackedEnsemble_BestOfFamily_AutoML_20181212_105540  0.788425   0.552145                0.323192  0.432625  0.187165
# XGBoost_1_AutoML_20181212_105540                     0.784651   0.55753                 0.325471  0.434949  0.189181
# XGBoost_grid_1_AutoML_20181212_105540_model_4        0.783523   0.557854                0.318819  0.435249  0.189441
# XGBoost_grid_1_AutoML_20181212_105540_model_3        0.783004   0.559613                0.325081  0.435708  0.189841
# XGBoost_2_AutoML_20181212_105540                     0.78136    0.55888                 0.347074  0.435907  0.190015
# XGBoost_3_AutoML_20181212_105540                     0.780847   0.559589                0.330739  0.43613   0.190209
# GBM_5_AutoML_20181212_105540                         0.780837   0.559903                0.340848  0.436191  0.190263
# GBM_2_AutoML_20181212_105540                         0.780036   0.559806                0.339926  0.436415  0.190458
# GBM_1_AutoML_20181212_105540                         0.779827   0.560857                0.335096  0.436616  0.190633
# GBM_3_AutoML_20181212_105540                         0.778669   0.56179                 0.325538  0.437189  0.191134
# XGBoost_grid_1_AutoML_20181212_105540_model_2        0.774411   0.575017                0.322811  0.4427    0.195984
# GBM_4_AutoML_20181212_105540                         0.771426   0.569712                0.33742   0.44107   0.194543
# GBM_grid_1_AutoML_20181212_105540_model_1            0.769752   0.572583                0.344331  0.442452  0.195764
# GBM_grid_1_AutoML_20181212_105540_model_2            0.754366   0.918567                0.355855  0.496638  0.246649
# DRF_1_AutoML_20181212_105540                         0.742892   0.595883                0.355403  0.452774  0.205004
# XRT_1_AutoML_20181212_105540                         0.742091   0.599346                0.356583  0.453117  0.205315
# DeepLearning_grid_1_AutoML_20181212_105540_model_2   0.741795   0.601497                0.368291  0.454904  0.206937
# XGBoost_grid_1_AutoML_20181212_105540_model_1        0.693554   0.620702                0.40588   0.465791  0.216961
# DeepLearning_1_AutoML_20181212_105540                0.69137    0.637954                0.409351  0.47178   0.222576
# DeepLearning_grid_1_AutoML_20181212_105540_model_1   0.690084   0.661794                0.418469  0.476635  0.227181
# GLM_grid_1_AutoML_20181212_105540_model_1            0.682648   0.63852                 0.397234  0.472683  0.223429
#
# [22 rows x 6 columns]

# The leader model is stored here
# aml.leader

AutoML progress: |
04:55:31.707: _train param, Dropping bad and constant columns: [rerun]

██
04:58:14.951: _train param, Dropping bad and constant columns: [rerun]

████████████████████████████████████████
05:05:02.509: _train param, Dropping bad and constant columns: [rerun]

█████████
05:09:29.19: _train param, Dropping bad and constant columns: [rerun]

██
05:10:54.512: _train param, Dropping bad and constant columns: [rerun]

██
05:13:37.65: _train param, Dropping bad and constant columns: [rerun]

██
05:16:35.777: _train param, Dropping bad and constant columns: [rerun]

██
05:19:38.306: _train param, Dropping bad and constant columns: [rerun]

█
05:22:48.255: _train param, Dropping bad and constant columns: [rerun]

█
05:26:40.615: _train param, Dropping bad and constant columns: [rerun]

█
05:29:59.409: _train param, Dropping bad and constant columns: [rerun]


05:32:05.875: _train param, Dropping bad and constant columns: [rerun]

█
06:26:30.1: _train param, Dropping unused co

key,value
Stacking strategy,cross_validation
Number of base models (used / total),19/25
# GBM base models (used / total),7/9
# XGBoost base models (used / total),5/8
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),5/5
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20230424_45531,0.013838,0.0241559,0.073564,0.00541166
StackedEnsemble_BestOfFamily_1_AutoML_1_20230424_45531,0.014247,0.0247664,0.0746377,0.00557079
GBM_1_AutoML_1_20230424_45531,0.0143203,0.0251269,0.074695,0.00557935
GBM_4_AutoML_1_20230424_45531,0.0144471,0.0257969,0.0750363,0.00563045
GBM_3_AutoML_1_20230424_45531,0.0147389,0.0261168,0.0756407,0.00572152
GBM_2_AutoML_1_20230424_45531,0.0151447,0.0266345,0.0764197,0.00583998
XGBoost_grid_1_AutoML_1_20230424_45531_model_4,0.0151901,0.0272567,0.0790016,0.00624125
XGBoost_grid_1_AutoML_1_20230424_45531_model_2,0.0153646,0.0274593,0.0791287,0.00626134
XGBoost_1_AutoML_1_20230424_45531,0.0154198,0.0279332,0.0798352,0.00637365
XGBoost_grid_1_AutoML_1_20230424_45531_model_1,0.0154405,0.0276824,0.0797768,0.00636434


In [ ]:
# The leader model is stored here
aml.leader

key,value
Stacking strategy,cross_validation
Number of base models (used / total),19/25
# GBM base models (used / total),7/9
# XGBoost base models (used / total),5/8
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),5/5
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [ ]:
# To generate predictions on a test set, you can make predictions
# directly on the `H2OAutoML` object or on the leader model
# object directly
# preds = aml.predict(test)

# or:
preds = aml.leader.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [ ]:
preds

predict,GALAXY,QSO,STAR
STAR,0.000229522,1.67353e-05,0.999754
STAR,0.00161655,8.45732e-05,0.998299
STAR,0.000459542,2.18801e-05,0.999519
QSO,0.00019632,0.999799,4.86782e-06
STAR,0.000160949,9.20525e-06,0.99983
QSO,0.0128432,0.987006,0.000150642
STAR,0.00087625,3.3945e-05,0.99909
STAR,0.000908605,9.99322e-05,0.998991
STAR,0.0012481,5.20993e-05,0.9987
GALAXY,0.998274,0.00163003,9.639e-05


In [ ]:
# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,mean_per_class_error,logloss,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_1_20230424_45531,0.013838,0.0241559,0.073564,0.00541166,1.16034e+06,0.178459,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_1_20230424_45531,0.014247,0.0247664,0.0746377,0.00557079,403312,0.042563,StackedEnsemble
GBM_1_AutoML_1_20230424_45531,0.0143203,0.0251269,0.074695,0.00557935,42058,0.019583,GBM
GBM_4_AutoML_1_20230424_45531,0.0144471,0.0257969,0.0750363,0.00563045,27518,0.01433,GBM
GBM_3_AutoML_1_20230424_45531,0.0147389,0.0261168,0.0756407,0.00572152,25401,0.014232,GBM
GBM_2_AutoML_1_20230424_45531,0.0151447,0.0266345,0.0764197,0.00583998,24901,0.016242,GBM
XGBoost_grid_1_AutoML_1_20230424_45531_model_4,0.0151901,0.0272567,0.0790016,0.00624125,11230,0.002412,XGBoost
XGBoost_grid_1_AutoML_1_20230424_45531_model_2,0.0153646,0.0274593,0.0791287,0.00626134,12664,0.002498,XGBoost
XGBoost_1_AutoML_1_20230424_45531,0.0154198,0.0279332,0.0798352,0.00637365,10557,0.002299,XGBoost
XGBoost_grid_1_AutoML_1_20230424_45531_model_1,0.0154405,0.0276824,0.0797768,0.00636434,11841,0.002295,XGBoost


## 1. Generalized Linear Model

In [ ]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [ ]:
#不是二分类 不能binary
glm_fit1 = H2OGeneralizedLinearEstimator(family='auto', model_id='glm_fit1',auc_type="MACRO_OVR")

In [ ]:
glm_fit1.train(x=x, y=y, training_frame=train)

glm Model Build progress: |

/usr/local/lib/python3.9/dist-packages/h2o/estimators/estimator_base.py:193: RuntimeWarning: Dropping bad and constant columns: [rerun]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: glm_fit1


GLM Model: summary
    family       link         regularization                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    multinomial  multinomial  Elastic Net (alpha = 0.5, lambda = 5.159E-4 )  66                            44                             33                      py_2_sid_baba

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.04272918783151669
RMSE: 0.20671039604121677
LogLoss: 0.17617220293755084
AUC: 0.9906932527395909
AUCPR: 0.9826854147667851
Null degrees of freedom: 350180
Residual degrees of freedom: 350134
Null deviance: 657449.322645345
Residual deviance: 123515.95657787687

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.986979
QSO vs Rest            QSO                                          0.991493
STAR vs Rest           STAR                                         0.993608
Macro OVR                                                           0.990693
Weighted OVR                                                        0.989805
Class GALAXY vs. QSO   GALAXY                QSO                    0.980503
Class GALAXY vs. STAR  GALAXY                STAR                   0.991866
Class QSO vs. STAR     QSO                   STAR                   0.997886
Macro OVO                                                           0.990085
Weighted OVO                                                        0.989581

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.987018
QSO vs Rest            QSO                                          0.977491
STAR vs Rest           STAR                                         0.983548
Macro OVR                                                           0.982685
Weighted OVR                                                        0.984777
Class GALAXY vs. QSO   GALAXY                QSO                    0.985312
Class GALAXY vs. STAR  GALAXY                STAR                   0.989376
Class QSO vs. STAR     QSO                   STAR                   0.996264
Macro OVO                                                           0.990317
Weighted OVO                                                        0.989649

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error      Rate
--------  -----  ------  ---------  ----------------
181031    386    7571    0.0421032  7,957 / 188,988
2993      34189  37      0.08141    3,030 / 37,219
2504      53     121417  0.0206253  2,557 / 123,974
186528    34628  129025  0.0386771  13,544 / 350,181

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.961323
2    0.997775
3    1

Scoring History: 
     timestamp            duration          iterations    negative_log_likelihood    objective            training_rmse        training_logloss     training_r2         training_classification_error    training_auc        training_pr_auc
---  -------------------  ----------------  ------------  -------------------------  -------------------  -------------------  -------------------  ------------------  -------------------------------  ------------------  ------------------
     2023-04-22 07:42:37  0.000 sec         0             328724.6613226725          0.938727861656322
     2023-04-22 07:42:48  10.252 sec        1  

In [ ]:
glm_fit2 = H2OGeneralizedLinearEstimator(family='auto', model_id='glm_fit2', lambda_search=True,auc_type="MACRO_OVR") 
glm_fit2.train(x=x, y=y, training_frame=train, validation_frame=valid)

glm Model Build progress: |

/usr/local/lib/python3.9/dist-packages/h2o/estimators/estimator_base.py:193: RuntimeWarning: Dropping bad and constant columns: [rerun]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: glm_fit2


GLM Model: summary
    family       link         regularization                                 lambda_search                                                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------------------------  ----------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    multinomial  multinomial  Elastic Net (alpha = 0.5, lambda = 5.159E-5 )  nlambda = 100, lambda.max = 0.5159, lambda.min = 5.159E-5, lambda.1se = -1.0  66                            54                             291                     py_2_sid_baba

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.021211302293837254
RMSE: 0.14564100485041037
LogLoss: 0.09882269697597831
AUC: 0.994534229531876
AUCPR: 0.9883070328083404
Null degrees of freedom: 350180
Residual degrees of freedom: 350124
Null deviance: 657449.322645345
Residual deviance: 69211.69027098341

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.992864
QSO vs Rest            QSO                                          0.993603
STAR vs Rest           STAR                                         0.997136
Macro OVR                                                           0.994534
Weighted OVR                                                        0.994455
Class GALAXY vs. QSO   GALAXY                QSO                    0.986908
Class GALAXY vs. STAR  GALAXY                STAR                   0.996154
Class QSO vs. STAR     QSO                   STAR                   0.999167
Macro OVO                                                           0.994076
Weighted OVO                                                        0.993861

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.993206
QSO vs Rest            QSO                                          0.980596
STAR vs Rest           STAR                                         0.991119
Macro OVR                                                           0.988307
Weighted OVR                                                        0.991127
Class GALAXY vs. QSO   GALAXY                QSO                    0.988377
Class GALAXY vs. STAR  GALAXY                STAR                   0.994528
Class QSO vs. STAR     QSO                   STAR                   0.998246
Macro OVO                                                           0.993717
Weighted OVO                                                        0.993397

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  ---------------
183720    508    4760    0.0278748   5,268 / 188,988
2402      34793  24      0.0651818   2,426 / 37,219
47        103    123824  0.00120993  150 / 123,974
186169    35404  128608  0.0223998   7,844 / 350,181

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.9776
2    0.999369
3    1

ModelMetricsMultinomialGLM: glm
** Reported on validation data. **

MSE: 0.021622072367122224
RMSE: 0.14704445711118194
LogLoss: 0.10001926791016588
AUC: 0.9943944763344336
AUCPR: 0.9883049009622087
Null degrees of freedom: 74944
Residual degrees of freedom: 74888
Null deviance: 140558.95456350024
Residual deviance: 14999.212071598427

Multinomial AUC values: 
type                   first_class_domain    seco

In [ ]:
glm_perf1 = glm_fit1.model_performance(test)
glm_perf2 = glm_fit2.model_performance(test)
# Print model performance
print (glm_perf1)
print (glm_perf2)
# Retreive test set AUC
print ('glm_perf1.auc:',glm_perf1.auc())
print ('glm_perf2.auc:',glm_perf2.auc())

ModelMetricsMultinomialGLM: glm
** Reported on test data. **

MSE: 0.043136253818787045
RMSE: 0.20769269081695446
LogLoss: 0.17725972164009796
AUC: 0.9904429364088528
AUCPR: 0.98242377522904
Null degrees of freedom: 74873
Residual degrees of freedom: 74827
Null deviance: 140839.3983557321
Residual deviance: 26580.240278591526

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.986942
QSO vs Rest            QSO                                          0.990596
STAR vs Rest           STAR                                         0.993791
Macro OVR                                                           0.990443
Weighted OVR                                                        0.989769
Class GALAXY vs. QSO   GALAXY                QSO                    0.97935
Class GALAXY vs. STAR  GALAXY       

## 2. random forest

In [ ]:
# Random Forest

# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator
# Initialize the RF estimator:
rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1,auc_type="MACRO_OVR")
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: rf_fit1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 150                         6.71551e+06            20           20           20            1096          7655          3559.05

ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.010352075396385116
RMSE: 0.10174514925236051
LogLoss: 0.05771728697684079
Mean Per-Class Error: 0.01979031332174949
AUC: 0.9972315925881158
AUCPR: 0.9935972585209892

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.997151
QSO vs Rest            QSO                                          0.995092
STAR vs Rest           STAR                                         0.999452
Macro OVR                                                           0.997232
Weighted OVR                                                        0.997747
Class GALAXY vs. QSO   GALAXY                QSO                    0.991829
Class GALAXY vs. STAR  GALAXY                STAR                   0.999245
Class QSO vs. STAR     QSO                   STAR                   0.999035
Macro OVO                                                           0.996703
Weighted OVO                                                        0.996801

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.996478
QSO vs Rest            QSO                                          0.985736
STAR vs Rest           STAR                                         0.998577
Macro OVR                                                           0.993597
Weighted OVR                                                        0.99608
Class GALAXY vs. QSO   GALAXY                QSO                    0.99166
Class GALAXY vs. STAR  GALAXY                STAR                   0.999093
Class QSO vs. STAR     QSO                   STAR                   0.998169
Macro OVO                                                           0.996307
Weighted OVO                                                        0.99648

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  ---------------
187445    478    1065    0.00816454  1,543 / 188,988
1816      35384  19      0.0493028   1,835 / 37,219
235       1      123738  0.00190362  236 / 123,974
189496    35863  124822  0.0103204   3,614 / 350,181

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.98968
2    0.999355
3    1

Scoring History: 
    timestamp            duration          number_of_trees    training_rmse    training_logloss    training_classification_error    training_auc    training_pr_auc
--  -------------------  ----------------  -----------------  ---------------  ------------------  -------------------------------  --------------  -----------------
    2023-04-22 08:04:31  0.047 sec         0                  nan              nan                 nan                              nan             nan
    2023-04-22 08:04:37  6.054 sec         1                  0.167534         0.787903            0.0269146                        0.978923        0.964611
    2023-04-22 08:04:44  13.268 sec        3                  0.142617         0.482095            0.0204136                        0.983953        0.971665
    2023-04-22

In [ ]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=100, seed=1,auc_type="MACRO_OVR")
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: rf_fit2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                300                         1.3537e+07             20           20           20            788           7655          3587.27

ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.01001941187742023
RMSE: 0.10009701233014016
LogLoss: 0.05173169102806518
Mean Per-Class Error: 0.01952642638729661
AUC: 0.99758181086637
AUCPR: 0.9942911342443074

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.997489
QSO vs Rest            QSO                                          0.995638
STAR vs Rest           STAR                                         0.999618
Macro OVR                                                           0.997582
Weighted OVR                                                        0.998046
Class GALAXY vs. QSO   GALAXY                QSO                    0.992506
Class GALAXY vs. STAR  GALAXY                STAR                   0.999472
Class QSO vs. STAR     QSO                   STAR                   0.999161
Macro OVO                                                           0.997046
Weighted OVO                                                        0.997151

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.996869
QSO vs Rest            QSO                                          0.986904
STAR vs Rest           STAR                                         0.9991
Macro OVR                                                           0.994291
Weighted OVR                                                        0.9966
Class GALAXY vs. QSO   GALAXY                QSO                    0.992404
Class GALAXY vs. STAR  GALAXY                STAR                   0.999395
Class QSO vs. STAR     QSO                   STAR                   0.998233
Macro OVO                                                           0.996677
Weighted OVO                                                        0.99687

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  ---------------
187490    459    1039    0.00792643  1,498 / 188,988
1798      35401  20      0.048846    1,818 / 37,219
224       0      123750  0.00180683  224 / 123,974
189512    35860  124809  0.0101091   3,540 / 350,181

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.989891
2    0.999489
3    1

Scoring History: 
    timestamp            duration          number_of_trees    training_rmse    training_logloss    training_classification_error    training_auc    training_pr_auc
--  -------------------  ----------------  -----------------  ---------------  ------------------  -------------------------------  --------------  -----------------
    2023-04-22 08:07:33  0.020 sec         0                  nan              nan                 nan                              nan             nan
    2023-04-22 08:07:37  4.485 sec         1                  0.167534         0.787903            0.0269146                        0.978923        0.964611
    2023-04-22 08:07:43  9.710 sec         3                  0.142617         0.482095            0.0204136                        0.983953        0.971665
    2023-04-22 08:

In [ ]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [ ]:
# Retreive test set AUC
print ('rf_perf1.auc:',rf_perf1.auc())
print ('rf_perf2.auc:',rf_perf2.auc())

rf_perf1.auc: 0.9975520887575917
rf_perf2.auc: 0.997783858330093


### 2.1 cross validate

In [ ]:
#cross-validate

#auc_type:https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/auc_type.html

rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=5,auc_type="MACRO_OVR")
rf_fit3.train(x=x, y=y, training_frame=data)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: rf_fit3


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 150                         8.63207e+06            20           20           20            1336          9565          4576.57

ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.010083126376560886
RMSE: 0.10041477170496822
LogLoss: 0.05286855609300052
Mean Per-Class Error: 0.019536243115757784
AUC: 0.9974640696699422
AUCPR: 0.9940006094806965

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.997335
QSO vs Rest            QSO                                          0.995487
STAR vs Rest           STAR                                         0.99957
Macro OVR                                                           0.997464
Weighted OVR                                                        0.997931
Class GALAXY vs. QSO   GALAXY                QSO                    0.992183
Class GALAXY vs. STAR  GALAXY                STAR                   0.999392
Class QSO vs. STAR     QSO                   STAR                   0.999196
Macro OVO                                                           0.996924
Weighted OVO                                                        0.99702

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.996679
QSO vs Rest            QSO                                          0.986408
STAR vs Rest           STAR                                         0.998914
Macro OVR                                                           0.994001
Weighted OVR                                                        0.996381
Class GALAXY vs. QSO   GALAXY                QSO                    0.992071
Class GALAXY vs. STAR  GALAXY                STAR                   0.999269
Class QSO vs. STAR     QSO                   STAR                   0.998364
Macro OVO                                                           0.996568
Weighted OVO                                                        0.996738

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  ---------------
267448    656    1493    0.00797116  2,149 / 269,597
2562      50507  37      0.0489399   2,599 / 53,106
301       0      176996  0.00169772  301 / 177,297
270311    51163  178526  0.010098    5,049 / 500,000

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.989902
2    0.9994
3    1

ModelMetricsMultinomial: drf
** Reported on cross-validation data. **

MSE: 0.009858537273174185
RMSE: 0.09929016705179917
LogLoss: 0.04964575354835398
Mean Per-Class Error: 0.01956158326916643
AUC: 0.9977430407250208
AUCPR: 0.9945971256130247

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.997613
QSO vs Rest            QSO                                          0.995973
STAR vs Rest           STAR                                         0.999643
Macro OVR                                                           0.997743
Weighted OVR                                                        0.998159
Class GALAXY vs. QSO   

In [ ]:
print ('rf_fit3.auc',rf_fit3.auc(xval=True))

rf_fit3.auc 0.9977430407250208


## 3. Gradient Boosting Machine

In [ ]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1,auc_type="MACRO_OVR")
gbm_fit1.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: gbm_fit1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 150                         64504                  5            5            5             14            32            29.5333

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.006586337833057864
RMSE: 0.08115625566188885
LogLoss: 0.032232819740704625
Mean Per-Class Error: 0.017606050502736493
AUC: 0.9981249497876478
AUCPR: 0.9948557319439546

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.998341
QSO vs Rest            QSO                                          0.99613
STAR vs Rest           STAR                                         0.999904
Macro OVR                                                           0.998125
Weighted OVR                                                        0.998659
Class GALAXY vs. QSO   GALAXY                QSO                    0.993441
Class GALAXY vs. STAR  GALAXY                STAR                   0.999881
Class QSO vs. STAR     QSO                   STAR                   0.999953
Macro OVO                                                           0.997758
Weighted OVO                                                        0.997818

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.998121
QSO vs Rest            QSO                                          0.986712
STAR vs Rest           STAR                                         0.999734
Macro OVR                                                           0.994856
Weighted OVR                                                        0.99748
Class GALAXY vs. QSO   GALAXY                QSO                    0.992451
Class GALAXY vs. STAR  GALAXY                STAR                   0.999837
Class QSO vs. STAR     QSO                   STAR                   0.999912
Macro OVO                                                           0.9974
Weighted OVO                                                        0.997469

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  ---------------
188208    572    208     0.00412725  780 / 188,988
1754      35444  21      0.0476907   1,775 / 37,219
124       0      123850  0.00100021  124 / 123,974
190086    36016  124079  0.00765033  2,679 / 350,181

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.99235
2    0.999937
3    1

Scoring History: 
    timestamp            duration          number_of_trees    training_rmse    training_logloss    training_classification_error    training_auc    training_pr_auc
--  -------------------  ----------------  -----------------  ---------------  ------------------  -------------------------------  --------------  -----------------
    2023-04-22 08:41:49  0.019 sec         0                  0.666667         1.09861             0.572521                         0.5             0.333333
    2023-04-22 08:41:56  7.288 sec         2                  0.537616         0.77177             0.0117054                        0.995424        0.988657
    2023-04-22 08:42:01  12.069 sec        5                  0.385351         0.487049            0.0112942                        0.996013        0.990281
    2

In [ ]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1,auc_type="MACRO_OVR")
gbm_fit2.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: gbm_fit2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    165                495                         176747                 0            5            4.37374       1             32            23.6545

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.004880248596482403
RMSE: 0.06985877608777871
LogLoss: 0.022656989735880997
Mean Per-Class Error: 0.013514769801960994
AUC: 0.9990138123902449
AUCPR: 0.9970313186994281

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.999122
QSO vs Rest            QSO                                          0.997971
STAR vs Rest           STAR                                         0.999948
Macro OVR                                                           0.999014
Weighted OVR                                                        0.999292
Class GALAXY vs. QSO   GALAXY                QSO                    0.996573
Class GALAXY vs. STAR  GALAXY                STAR                   0.999928
Class QSO vs. STAR     QSO                   STAR                   0.999982
Macro OVO                                                           0.998827
Weighted OVO                                                        0.998857

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.999057
QSO vs Rest            QSO                                          0.992182
STAR vs Rest           STAR                                         0.999855
Macro OVR                                                           0.997031
Weighted OVR                                                        0.998609
Class GALAXY vs. QSO   GALAXY                QSO                    0.995654
Class GALAXY vs. STAR  GALAXY                STAR                   0.999901
Class QSO vs. STAR     QSO                   STAR                   0.999958
Macro OVO                                                           0.998504
Weighted OVO                                                        0.998542

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error        Rate
--------  -----  ------  -----------  ---------------
188464    371    153     0.00277266   524 / 188,988
1364      35845  10      0.0369166    1,374 / 37,219
103       3      123868  0.000855018  106 / 123,974
189931    36219  124031  0.00572275   2,004 / 350,181

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.994277
2    0.999946
3    1

Scoring History: 
     timestamp            duration          number_of_trees    training_rmse        training_logloss      training_classification_error    training_auc        training_pr_auc
---  -------------------  ----------------  -----------------  -------------------  --------------------  -------------------------------  ------------------  ------------------
     2023-04-22 08:43:27  0.009 sec         0.0                0.6666666666669901   1.098612288667946     0.5725210676764302               0.5                 0.3333333333333334
     2023-04-22 08:43:29  2.488 sec         1.0                0.5992926621787135   0.9146840378172698    0.011965240832598                0.9953680872709922  0.9885592458408348
     2023-04-22 08:43:33  6.496 sec         3.0                0.48152543603510195  0.

In [ ]:
# Now let's use early stopping to find optimal ntrees

gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=5,     #used for early stopping
                                        stopping_rounds=3,         #used for early stopping
                                        stopping_metric='AUC',     #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1,
                                       auc_type="MACRO_OVR")

# The use of a validation_frame is recommended with using early stopping
gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: gbm_fit3


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    40                 120                         52422                  5            5            5             23            32            30.0833

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.007007971373382679
RMSE: 0.08371362716656518
LogLoss: 0.03678050135591256
Mean Per-Class Error: 0.018395980492409186
AUC: 0.9978081897438419
AUCPR: 0.9942217287999119

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.998104
QSO vs Rest            QSO                                          0.995452
STAR vs Rest           STAR                                         0.999869
Macro OVR                                                           0.997808
Weighted OVR                                                        0.998447
Class GALAXY vs. QSO   GALAXY                QSO                    0.992454
Class GALAXY vs. STAR  GALAXY                STAR                   0.999845
Class QSO vs. STAR     QSO                   STAR                   0.999899
Macro OVO                                                           0.997399
Weighted OVO                                                        0.99747

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.997841
QSO vs Rest            QSO                                          0.985183
STAR vs Rest           STAR                                         0.999641
Macro OVR                                                           0.994222
Weighted OVR                                                        0.997133
Class GALAXY vs. QSO   GALAXY                QSO                    0.991572
Class GALAXY vs. STAR  GALAXY                STAR                   0.999788
Class QSO vs. STAR     QSO                   STAR                   0.9998
Macro OVO                                                           0.997053
Weighted OVO                                                        0.997137

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  ---------------
188240    538    210     0.00395792  748 / 188,988
1844      35354  21      0.0501088   1,865 / 37,219
139       0      123835  0.0011212   139 / 123,974
190223    35892  124066  0.00785879  2,752 / 350,181

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.992141
2    0.999934
3    1

ModelMetricsMultinomial: gbm
** Reported on validation data. **

MSE: 0.007694631578643392
RMSE: 0.0877190491207206
LogLoss: 0.03915152721031102
Mean Per-Class Error: 0.020528433966561226
AUC: 0.997493278693146
AUCPR: 0.9936727517548407

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.998008
QSO vs Rest            QSO                                          0.99461
STAR vs Rest           STAR                                         0.999862
Macro OVR                                                           0.997493
Weighted OVR                                                        0.998311
Class GALAXY vs. QSO   GALA

### 3.1 compare model performance

In [ ]:
gbm_perf1 = gbm_fit1.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
gbm_perf3 = gbm_fit3.model_performance(test)

In [ ]:
# Retreive test set AUC
print("AUC")
print ('gbm_perf1',gbm_perf1.auc())
print ('gbm_perf2',gbm_perf2.auc())
print ('gbm_perf3',gbm_perf3.auc())

AUC
gbm_perf1 0.9978017328662036
gbm_perf2 0.9985086500241785
gbm_perf3 0.9975005213319008


In [ ]:
#scoring history
gbm_fit2.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2023-04-22 08:43:27,0.009 sec,0.0,0.666667,1.098612,0.572521,0.500000,0.333333
1,,2023-04-22 08:43:29,2.488 sec,1.0,0.599293,0.914684,0.011965,0.995368,0.988559
2,,2023-04-22 08:43:33,6.496 sec,3.0,0.481525,0.657437,0.011691,0.995673,0.989637
3,,2023-04-22 08:43:39,12.646 sec,6.0,0.344803,0.422719,0.011220,0.996020,0.990582
4,,2023-04-22 08:43:53,25.805 sec,14.0,0.156014,0.157608,0.009195,0.996955,0.992196
5,,2023-04-22 08:44:11,43.815 sec,24.0,0.096279,0.065809,0.008490,0.997378,0.993178
6,,2023-04-22 08:44:20,53.114 sec,30.0,0.089089,0.048650,0.008207,0.997490,0.993492
7,,2023-04-22 08:44:30,1 min 3.251 sec,35.0,0.085774,0.041124,0.007993,0.997665,0.993848
8,,2023-04-22 08:44:40,1 min 13.180 sec,40.0,0.083714,0.036781,0.007859,0.997808,0.994222
9,,2023-04-22 08:44:54,1 min 27.279 sec,48.0,0.081537,0.032827,0.007667,0.998086,0.994760


In [ ]:
gbm_fit3.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2023-04-22 08:48:01,0.015 sec,0.0,0.666667,1.098612,0.572521,0.500000,0.333333,0.666667,1.098612,0.574421,0.500000,0.333333
1,,2023-04-22 08:48:11,10.029 sec,5.0,0.385351,0.487049,0.011294,0.996013,0.990281,0.385626,0.487539,0.011915,0.995596,0.989116
2,,2023-04-22 08:48:19,17.844 sec,10.0,0.224691,0.250241,0.009618,0.996663,0.991656,0.225428,0.251099,0.010074,0.996259,0.990665
3,,2023-04-22 08:48:30,28.737 sec,15.0,0.144642,0.141787,0.009124,0.997024,0.992327,0.146136,0.143027,0.009687,0.996690,0.991545
4,,2023-04-22 08:48:41,39.673 sec,20.0,0.108728,0.088439,0.008693,0.997282,0.992843,0.111090,0.090031,0.009407,0.996955,0.992139
5,,2023-04-22 08:48:49,48.142 sec,25.0,0.094509,0.061918,0.008447,0.997413,0.993240,0.097561,0.063807,0.009153,0.997155,0.992705
6,,2023-04-22 08:49:00,58.928 sec,30.0,0.089089,0.048650,0.008207,0.997490,0.993492,0.092552,0.050717,0.009020,0.997266,0.993042
7,,2023-04-22 08:49:10,1 min 9.541 sec,35.0,0.085774,0.041124,0.007993,0.997665,0.993848,0.089575,0.043372,0.008820,0.997401,0.993383
8,,2023-04-22 08:49:19,1 min 18.036 sec,40.0,0.083714,0.036781,0.007859,0.997808,0.994222,0.087719,0.039152,0.008673,0.997493,0.993673


## 4. Deep Learning

In [ ]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
# Initialize and train the DL estimator:
dl_fit1 = H2ODeepLearningEstimator(model_id='dl_fit1', seed=1,auc_type="MACRO_OVR")
dl_fit1.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |

/usr/local/lib/python3.9/dist-packages/h2o/estimators/estimator_base.py:193: RuntimeWarning: Dropping bad and constant columns: [rerun]
  warnings.warn(mesg["message"], RuntimeWarning)


█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: dl_fit1


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 45,203 weights/biases, 541.5 KB, 496,840 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight           weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  --------------------  -------------------  --------------------  --------------------
    1        21       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.007526459264817753  0.024424932897090912  0.0         0.02011167863108702   0.16163551807403564  0.21358471941319102   0.2029094099998474
    3        200      Rectifier  0.0        0.0   0.0   0.03579666889890214   0.0967153012752533    0.0         0.001959588778293072  0.10270985960960388  0.9487034245077889    0.3307666778564453
    4        3        Softmax               0.0   0.0   0.05428052926142603   0.19702744483947754   0.0         -0.07543051661419062  0.4278024435043335   -0.01821171996276384  0.026228979229927063

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.013163668171975123
RMSE: 0.11473302999561688
LogLoss: 0.06607495795491085
Mean Per-Class Error: 0.020911679026207306
AUC: 0.9969431684768231
AUCPR: 0.9926946911060401

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.996061
QSO vs Rest            QSO                                          0.996559
STAR vs Rest           STAR                                         0.998209
Macro OVR                                                           0.996943
Weighted OVR                                                        0.996875
Class GALAXY vs. QSO   GALAXY                QSO                    0.992946
Class GALAXY vs. STAR  GALAXY                STAR                   0.997682
Class QSO vs. STAR     QSO                   STAR                   0.999907
Macro OVO                                                           0.996845
Weighted OVO                                                        0.996666

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.99664
QSO vs Rest            QSO                                          0.987507
STAR vs Rest           STAR                                         0.993936
Macro OVR                                                           0.992695
Weighted OVR                                                        0.99468
Class GALAXY vs. QSO   GALAXY                QSO                    0.992689
Class GALAXY vs. STAR  GALAXY                STAR                   0.996358
Class QSO vs. STAR     QSO                   STAR                   0.999911
Macro OVO                                                           0.996319
Weighted OVO                                                        0.995995

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  -----------
5190      13     87      0.0189036   100 / 5,290
46        1039   0       0.0423963   46 / 1,085
4         1      3479    0.00143513  5 / 3,484
5240      1053   3566    0.015316    151 / 9,859

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.984684
2    0.999797
3    1

Scoring History: 
     timestamp            duration          training_speed    epochs                iterations    samples 

In [ ]:
dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[10,10], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1,
                                   auc_type="MACRO_OVR")
dl_fit2.train(x=x, y=y, training_frame=train)

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: dl_fit2


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 363 weights/biases, 11.6 KB, 7,029,477 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight          weight_rms          mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  -------------------  ------------------  -------------------  ------------------
    1        21       Input      0.0
    2        10       Rectifier  0.0        0.0   0.0   0.002115804598031987  0.006088294088840485  0.0         -0.0944486396071235  0.5199356079101562  0.2623336435396352   1.553999900817871
    3        10       Rectifier  0.0        0.0   0.0   0.02769995972581455   0.07326966524124146   0.0         0.35459566449280827  1.6361422538757324  0.9638469237461658   2.72934627532959
    4        3        Softmax               0.0   0.0   0.00361754732607551   0.005885422229766846  0.0         -0.6001572844882807  6.098482131958008   0.13676335987514412  0.4208277463912964

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.009197547064454402
RMSE: 0.09590384280337469
LogLoss: 0.05120116755663013
Mean Per-Class Error: 0.0164398536377443
AUC: 0.996879114690597
AUCPR: 0.9920653966097331

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.99622
QSO vs Rest            QSO                                          0.996036
STAR vs Rest           STAR                                         0.998381
Macro OVR                                                           0.996879
Weighted OVR                                                        0.996964
Class GALAXY vs. QSO   GALAXY                QSO                    0.993377
Class GALAXY vs. STAR  GALAXY                STAR                   0.997693
Class QSO vs. STAR     QSO                   STAR                   0.999429
Macro OVO                                                           0.996833
Weighted OVO                                                        0.9967

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.995837
QSO vs Rest            QSO                                          0.987823
STAR vs Rest           STAR                                         0.992536
Macro OVR                                                           0.992065
Weighted OVR                                                        0.993788
Class GALAXY vs. QSO   GALAXY                QSO                    0.992623
Class GALAXY vs. STAR  GALAXY                STAR                   0.995422
Class QSO vs. STAR     QSO                   STAR                   0.999391
Macro OVO                                                           0.995812
Weighted OVO                                                        0.995437

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error       Rate
--------  -----  ------  ----------  -----------
5225      22     43      0.0122873   65 / 5,290
38        1047   0       0.035023    38 / 1,085
6         1      3477    0.00200918  7 / 3,484
5269      1070   3520    0.0111573   110 / 9,859

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.988843
2    0.999899
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples      training_rmse    training_logloss    traini

In [ ]:
dl_fit3 = H2ODeepLearningEstimator(model_id='dl_fit3', 
                                   epochs=20, 
                                   hidden=[10,10],
                                   score_interval=1,          #used for early stopping
                                   stopping_rounds=3,         #used for early stopping
                                   stopping_metric='AUC',     #used for early stopping
                                   stopping_tolerance=0.0005, #used for early stopping
                                   seed=1,
                                  auc_type="MACRO_OVR")
dl_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: dl_fit3


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 363 weights/biases, 13.2 KB, 7,029,477 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms          mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  ------------------  -------------------  -------------------
    1        21       Input      0.0
    2        10       Rectifier  0.0        0.0   0.0   0.0006222407134910804  0.0017802207730710506  0.0         -0.028936644181036777  0.5123775005340576  0.03281153298326996  0.9636871814727783
    3        10       Rectifier  0.0        0.0   0.0   0.004200513731836964   0.010144621133804321   0.0         0.3928414483903907     1.161825180053711   1.48345603954686     2.0273704528808594
    4        3        Softmax               0.0   0.0   0.0037205448192253244  0.008295703679323196   0.0         -0.346097332239151     3.987825393676758   0.15777030425837904  0.42317450046539307

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.009839857496757153
RMSE: 0.0991960558528269
LogLoss: 0.054098031333650404
Mean Per-Class Error: 0.014715614547557342
AUC: 0.9972774061340255
AUCPR: 0.992074889413945

Multinomial AUC values: 
type                   first_class_domain    second_class_domain    auc
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.996175
QSO vs Rest            QSO                                          0.997658
STAR vs Rest           STAR                                         0.998
Macro OVR                                                           0.997277
Weighted OVR                                                        0.996983
Class GALAXY vs. QSO   GALAXY                QSO                    0.995095
Class GALAXY vs. STAR  GALAXY                STAR                   0.997215
Class QSO vs. STAR     QSO                   STAR                   0.999986
Macro OVO                                                           0.997432
Weighted OVO                                                        0.997172

Multinomial auc_pr values: 
type                   first_class_domain    second_class_domain    auc_pr
---------------------  --------------------  ---------------------  --------
GALAXY vs Rest         GALAXY                                       0.997284
QSO vs Rest            QSO                                          0.988019
STAR vs Rest           STAR                                         0.990921
Macro OVR                                                           0.992075
Weighted OVR                                                        0.994016
Class GALAXY vs. QSO   GALAXY                QSO                    0.993277
Class GALAXY vs. STAR  GALAXY                STAR                   0.994746
Class QSO vs. STAR     QSO                   STAR                   0.999953
Macro OVO                                                           0.995992
Weighted OVO                                                        0.995478

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
GALAXY    QSO    STAR    Error        Rate
--------  -----  ------  -----------  -----------
5214      16     60      0.0143667    76 / 5,290
32        1053   0       0.0294931    32 / 1,085
0         1      3483    0.000287026  1 / 3,484
5246      1070   3543    0.0110559    109 / 9,859

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.988944
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.011266148175339394
RMSE: 0.10614211310945056
L

### 4.1 compare erformance

In [ ]:
dl_perf1 = dl_fit1.model_performance(test)
dl_perf2 = dl_fit2.model_performance(test)
dl_perf3 = dl_fit3.model_performance(test)

In [ ]:
# Retreive test set AUC
print("AUC")
print ('dl_perf1',dl_perf1.auc())
print ('dl_perf2',dl_perf2.auc())
print ('dl_perf3',dl_perf3.auc())

AUC
dl_perf1 0.9957299885211475
dl_perf2 0.9963710084676566
dl_perf3 0.9964908678024497


In [ ]:
dl_fit3.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_r2,validation_classification_error,validation_auc,validation_pr_auc
0,,2023-04-22 08:54:24,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2023-04-22 08:54:24,1.547 sec,122333 obs/sec,0.178865,1,62635.0,0.159144,0.116304,0.970426,0.028908,0.993988,0.987640,0.169154,0.129073,0.966787,0.032250,0.992058,0.983565
2,,2023-04-22 08:54:33,11.278 sec,128848 obs/sec,3.042572,17,1065451.0,0.113352,0.064149,0.984997,0.015215,0.996705,0.991492,0.119535,0.069562,0.983414,0.016906,0.995666,0.990039
3,,2023-04-22 08:54:53,30.175 sec,127766 obs/sec,9.503137,53,3327818.0,0.099196,0.054098,0.988510,0.011056,0.997277,0.992075,0.106142,0.059897,0.986923,0.013036,0.996377,0.991119
4,,2023-04-22 08:55:04,42.878 sec,139255 obs/sec,14.518064,81,5083950.0,0.095643,0.052813,0.989319,0.011056,0.997101,0.989690,0.105380,0.059049,0.987110,0.012836,0.996358,0.990025
5,,2023-04-22 08:55:17,55.137 sec,150318 obs/sec,20.073839,112,7029477.0,0.098857,0.054147,0.988588,0.010244,0.997103,0.993739,0.106764,0.061517,0.986769,0.012783,0.996246,0.991484
6,,2023-04-22 08:55:19,57.284 sec,150308 obs/sec,20.073839,112,7029477.0,0.099196,0.054098,0.988510,0.011056,0.997277,0.992075,0.106142,0.059897,0.986923,0.013036,0.996377,0.991119


## 5. Naive Bayes

In [ ]:
# Import H2O NB:
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator

In [ ]:
# Initialize and train the NB estimator:

nb_fit1 = H2ONaiveBayesEstimator(model_id='nb_fit1',auc_type="MACRO_OVR")
nb_fit1.train(x=x, y=y, training_frame=train)

naivebayes Model Build progress: |

/usr/local/lib/python3.9/dist-packages/h2o/estimators/estimator_base.py:193: RuntimeWarning: Dropping bad and constant columns: [rerun]
  warnings.warn(mesg["message"], RuntimeWarning)


███████████████████████████████████████████████| (done) 100%


,number_of_response_levels,min_apriori_probability,max_apriori_probability
,3.0,0.1062850,0.5396866
type,first_class_domain,second_class_domain,auc
GALAXY vs Rest,GALAXY,None,0.9110630
QSO vs Rest,QSO,None,0.9566931
STAR vs Rest,STAR,None,0.9596470
Macro OVR,None,None,0.9424677
Weighted OVR,None,None,0.9331129
Class GALAXY vs. QSO,GALAXY,QSO,0.9648219
Class GALAXY vs. STAR,GALAXY,STAR,0.9261342
Class QSO vs. STAR,QSO,STAR,0.9576496


In [ ]:
nb_fit2 = H2ONaiveBayesEstimator(model_id='nb_fit2', laplace=6,auc_type="MACRO_OVR")
nb_fit2.train(x=x, y=y, training_frame=train)

naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%


,number_of_response_levels,min_apriori_probability,max_apriori_probability
,3.0,0.1062967,0.5396760
type,first_class_domain,second_class_domain,auc
GALAXY vs Rest,GALAXY,None,0.9110549
QSO vs Rest,QSO,None,0.9566970
STAR vs Rest,STAR,None,0.9596670
Macro OVR,None,None,0.9424730
Weighted OVR,None,None,0.9331160
Class GALAXY vs. QSO,GALAXY,QSO,0.9648288
Class GALAXY vs. STAR,GALAXY,STAR,0.9261657
Class QSO vs. STAR,QSO,STAR,0.9576084


In [ ]:
# performance
nb_perf1 = nb_fit1.model_performance(test)
nb_perf2 = nb_fit2.model_performance(test)

In [ ]:
# Retreive test set AUC
print ('nb_perf1',nb_perf1.auc())
print ('nb_perf2',nb_perf2.auc())

nb_perf1 0.943663073836246
nb_perf2 0.9436283420339472


## END

In [ ]:
h2o.shutdown(prompt=False)

H2O session _sid_baba closed.


<ipython-input-46-02ff3806b010>:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)
